In [1]:
# RNN LSTM 主要面对 语音 文本 这样的序列化的问题 但是同样可以用在图片分类问题

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# from tensorflow.contrib import rnn 
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# 每批次训练数据大小 和 批次大小
batch_size = 50
n_batch = mnist.train.num_examples // batch_size

# 输入图片是28×28
depth = 28                 # 词向量维度 输入层有28个神经元 刚好对应数字图片一行有28个像素
max_time = 28              # 每一个时序 28是最大时间步长 一共28行 相当于输入28次
lstm_size = 100            # 隐藏层有100个“神经元” 它不是简单的神经元 它是一个block
n_classes = 10             # 共10个分类

# none表示第一个维度可以是任意的长度
# 给输入数据预留位置
x = tf.placeholder(tf.float32, shape = [None,784])
y = tf.placeholder(tf.float32, shape = [None,10])

# 初始化权值 权值矩阵
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1)) # [隐藏层数量，分类个数]
# 初始化偏置 偏置值矩阵
biases = tf.Variable(tf.constant(0.1,shape=[n_classes])) # 10个分类就10个偏置


# ？？LSTM的输入层到隐藏层没有权值和偏置？？？
# 如果不给 tf会自动分配


# 定义RNN网络
def RNN(X,weights,biases):
    # inputs的格式需要是：[batch_size,max_time,n_inputs]
    inputs = tf.reshape(X,[-1,max_time,depth])
    # 定义LSTM基本CELL
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
#     lstm_cell = rnn.BasicLSTMCell(lstm_size)

    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
    # outputs 记录着（输入28次）每一次输出的结果 就是ht 也是yt
    # final_state 记录着（输入28次）最后一次的输出结果

    # final_state[0]是cell_state
    # final_state[1]是hidden_state
    results = tf.nn.softmax(tf.matmul(final_state[1],weights) + biases)
    return results




# x一进来就先被reshape了
# 这里的batch_size是50 那就是说一次性能进来50张图 那None就是50 x的shape就是[50,784]
# -1的位置也就是50了 变换完就成了[batch_size,max_time,n_inputs]格式[50,28,28]
# 为什么要变成这样的格式呢 看在哪儿调用了inputs
# tf.nn.dynamic_rnn()这个函数 的inputs 必须是这样的格式[批次大小，序列长度，序列中每次传入的数据]
# LSTM的CELL构建 BasicLSTMCell可改 这里是定义了100个LSTM单元

# 执行tf.nn.dynamic_rnn （隐藏cell，输入数据，定义类型）
# 得到两个返回值 outputs和final_state 
# final_state：这个列表包括三个维度[state,batch_size,cell.state_size]
# state包含两个元素 state[0]是cell_state
#                   state[1]是hidden_state
# batch_size就是之前定义的批次中有多少个样本
# cell.state_size是隐藏单元的个数 就是创建cell时tf.contrib.rnn.BasicLSTMCell(lstm_size)的lstm_size

#  outputs: The RNN output `Tensor`.
#     If time_major == False (default), this will be a `Tensor` shaped:
#       `[batch_size, max_time, cell.output_size]`.
#     If time_major == True, this will be a `Tensor` shaped:
#       `[max_time, batch_size, cell.output_size]`.

#     Note, if `cell.output_size` is a (possibly nested) tuple of integers
#     or `TensorShape` objects, then `outputs` will be a tuple having the
#     same structure as `cell.output_size`, containing Tensors having shapes
#     corresponding to the shape data in `cell.output_size`.

#   state: The final state.  If `cell.state_size` is an int, this
#     will be shaped `[batch_size, cell.state_size]`.  If it is a
#     `TensorShape`, this will be shaped `[batch_size] + cell.state_size`.
#     If it is a (possibly nested) tuple of ints or `TensorShape`, this will
#     be a tuple having the corresponding shapes. If cells are `LSTMCells`
#     `state` will be a tuple containing a `LSTMStateTuple` for each cell.


# tf.nn.dynamic_rnn(
#     cell,
# 自己定义的LSTM的细胞单元,如是convLSTM,自己写也可以。
#     inputs,
# 一个三维的变量,[batchsize,time_step,input_size],搭配time_major=False。
# 其中batch_size表示batch的大小。time_steps序列长度，input_size输入数据单个序列单个时间维度上固有的长度。
# 这里还补充一点,就是叫dynamic的原因,就是输入数据的time_step不一定要相同
# 如果长短不一,会自动跟短的补0,但是处理时候,不会处理0,在0前面就截止了.
# 这就是dynamic对比static的好处.
# time_major
# If true,   these Tensors must be shaped [max_time, batch_size, depth].
# If false, these Tensors must be shaped `[batch_size, max_time, depth] 
#     sequence_length=None,
#     initial_state=None,
#     dtype=None,
#     parallel_iterations=None,
#     swap_memory=False,
#     time_major=False,
#     scope=None
# )




# 预测值 计算RNN的返回结果
prediction = RNN(x, weights, biases)  # x:一个批次的训练数据  权值  偏置值  （不传也可以 因为这里是全局变量）
# 交叉熵代价函数
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = prediction))
# 优化器
train = tf.train.AdamOptimizer(0.0001).minimize(loss)
# 定义求准确率的方法
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
# 初始化变量
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(101):
        for batch in range(n_batch):
            batch_data,batch_tag = mnist.train.next_batch(batch_size)
            sess.run(train,feed_dict = {x:batch_data,y:batch_tag})
        if epoch%10==0:
            train_accuracy_rate = sess.run(accuracy,feed_dict = {x:mnist.train.images,y:mnist.train.labels})# 训练集正确率
            test_accuracy_rate = sess.run(accuracy,feed_dict = {x:mnist.test.images,y:mnist.test.labels})   # 测试集正确率
            print('第' + str(epoch + 1) + '次，train准确率 ' + str(train_accuracy_rate) + ' test准确率 ' + str(test_accuracy_rate))
tf.logging.set_verbosity(old_v)


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


InternalError: Blas GEMM launch failed : a.shape=(50, 128), b.shape=(128, 400), m=50, n=400, k=128
	 [[node rnn/while/basic_lstm_cell/MatMul (defined at <ipython-input-2-b7fe08cdbbc1>:37) ]]

Caused by op 'rnn/while/basic_lstm_cell/MatMul', defined at:
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File